## Consolidacion de archivos 

### Importacion de librerias y carga de datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import os
import sys
import re
import duckdb

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [2]:
## Primer archivo 2017-2025
df_desnutricion = pd.read_excel("../data/raw/DATA PROYECTO BREIT.xlsx", sheet_name="DESNUTRICION")
df_desarrollo = pd.read_excel("../data/raw/DATA PROYECTO BREIT.xlsx", sheet_name="DESARROLLO")
df_diccionario = pd.read_excel("../data/raw/DATA PROYECTO BREIT.xlsx", sheet_name="DICCIONARIO DE DATOS")

## Segundo archivo 2009-2016
df_desnutricion_old = pd.read_excel("../data/raw/DATA 2009-2016.xlsx", sheet_name="NUTRICION")
df_desarrollo_old = pd.read_excel("../data/raw/DATA 2009-2016.xlsx", sheet_name="DESARROLLO")

### Descripcion general de datasets

In [3]:
print("Shape df_desnutricion:", df_desnutricion.shape)
print("Shape df_desarrollo:", df_desarrollo.shape)
print("Shape df_diccionario:", df_diccionario.shape)
print("Shape df_desnutricion_old:", df_desnutricion_old.shape)
print("Shape df_desarrollo_old:", df_desarrollo_old.shape)

Shape df_desnutricion: (257178, 27)
Shape df_desarrollo: (257178, 18)
Shape df_diccionario: (35, 5)
Shape df_desnutricion_old: (197832, 28)
Shape df_desarrollo_old: (197832, 19)


In [5]:
df_desnutricion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257178 entries, 0 to 257177
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Fecha                   257178 non-null  datetime64[ns]
 1   Nº_HC                   257169 non-null  float64       
 2   Tipo_Paciente           257178 non-null  object        
 3   Categoría               257178 non-null  object        
 4   Sexo                    257178 non-null  object        
 5   Edad                    257178 non-null  object        
 6   Nº_Control              257177 non-null  float64       
 7   Peso                    257171 non-null  object        
 8   Talla                   257160 non-null  object        
 9   CabPC                   214658 non-null  object        
 10  Diag_Nacimiento         257178 non-null  object        
 11  P/T                     209486 non-null  object        
 12  T/E                     209489

In [6]:
df_desarrollo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257178 entries, 0 to 257177
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Fecha             257178 non-null  datetime64[ns]
 1   Nº_HC             257169 non-null  float64       
 2   Tipo_Paciente     257178 non-null  object        
 3   Categoría         257178 non-null  object        
 4   Sexo              257178 non-null  object        
 5   Edad              257178 non-null  object        
 6   Nº_Control        257177 non-null  float64       
 7   Peso              257171 non-null  object        
 8   Talla             257160 non-null  object        
 9   CabPC             214658 non-null  object        
 10  Diag_Nacimiento   257178 non-null  object        
 11  (M) - FG          244276 non-null  object        
 12  (M) - FF          126452 non-null  object        
 13  (C) - Cog         244273 non-null  object        
 14  (L) 

In [7]:
df_diccionario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Nombre del campo  34 non-null     object
 1   Descripción       33 non-null     object
 2   Tipo de Dato      33 non-null     object
 3   Ejemplo           33 non-null     object
 4   Opciones válidas  33 non-null     object
dtypes: object(5)
memory usage: 1.5+ KB


In [10]:
## DICCIONARIO
df_diccionario.head(5)

,Nombre del campo,Descripción,Tipo de Dato,Ejemplo,Opciones válidas
0,Fecha,Fecha de atención,Fecha/Hora,2025-01-10 00:00:00,--
1,Nombre_Paciente,Nombre del niño(a) atendido(a),Texto Corto,Sanchez Diaz Madison Aurora,--
2,Nº_HC,Historia clínica asignada,Número,88025,--
3,Tipo_Paciente,Tipo de atención del paciente,Texto Corto,CRED,"CRED, Control RN"
4,Categoría,Continuidad del paciente en sus atenciones,Texto Corto,Continuador,"Nuevo, Continuador"


In [8]:
df_diccionario[df_diccionario['Nombre del campo'] == 'CabPC']

,Nombre del campo,Descripción,Tipo de Dato,Ejemplo,Opciones válidas
10,CabPC,Perimetro cefálico,Texto Corto,45.2,--


In [12]:
df_desnutricion_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197832 entries, 0 to 197831
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Fecha                   197832 non-null  datetime64[ns]
 1   Nombre_Paciente         197832 non-null  object        
 2   Nº_HC                   197809 non-null  float64       
 3   Tipo_Paciente           197832 non-null  object        
 4   Categoría               197832 non-null  object        
 5   Sexo                    197832 non-null  object        
 6   Edad                    197832 non-null  object        
 7   Nº_Control              197832 non-null  int64         
 8   Peso                    197830 non-null  object        
 9   Talla                   197827 non-null  object        
 10  CabPC                   63984 non-null   object        
 11  Diag_Nacimiento         197832 non-null  object        
 12  P/T                     197818

In [13]:
df_desarrollo_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197832 entries, 0 to 197831
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Fecha             197832 non-null  datetime64[ns]
 1   Nombre_Paciente   197832 non-null  object        
 2   Nº_HC             197737 non-null  float64       
 3   Tipo_Paciente     197832 non-null  object        
 4   Categoría         197832 non-null  object        
 5   Sexo              197832 non-null  object        
 6   Edad              197832 non-null  object        
 7   Nº_Control        197832 non-null  int64         
 8   Peso              197830 non-null  object        
 9   Talla             197827 non-null  object        
 10  CabPC             63984 non-null   object        
 11  Diag_Nacimiento   197832 non-null  object        
 12  (M) - FG          197440 non-null  object        
 13  (M) - FF          1 non-null       object        
 14  (C) 

In [14]:
df_desnutricion.head()

,Fecha,Nº_HC,Tipo_Paciente,Categoría,Sexo,Edad,Nº_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera
0,2019-05-09,84898.0,CRED,Continuador,F,2m,2.0,4.945,55,38.4,PTIN,+1,<-1,-1,GIP-GIT,Riesgo T/E,CN,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Papá,4
1,2019-06-07,84898.0,CRED,Continuador,F,3m,3.0,5.880,58.5,40,PTIN,+1,<-1,-1,GIP-GIT,Riesgo T/E,CA,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2
2,2019-07-09,84898.0,CRED,Continuador,F,4m,4.0,6.070,59.6,--,PTIN,+1,-2,<-1,GIP-GIT,Riesgo P/E,CA,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2
3,2019-08-07,84898.0,CRED,Continuador,F,5m,5.0,6.945,60.8,42.5,PTIN,>+1,<-2,-1,GAP-GIT,Riesgo T/E,CA,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2
4,2019-09-09,84898.0,CRED,Continuador,F,6m,6.0,7.705,63.1,43.4,PTIN,>+1,-2,-1,GAP-GIT,Riesgo T/E,CA,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2


In [15]:
df_desarrollo.head()

,Fecha,Nº_HC,Tipo_Paciente,Categoría,Sexo,Edad,Nº_Control,Peso,Talla,CabPC,Diag_Nacimiento,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc,Acompaña_control,Enfermera
0,2019-05-09,84898.0,CRED,Continuador,F,2m,2.0,4.945,55,38.4,PTIN,EP,NaN,SI,SI,SI,Papá,4
1,2019-06-07,84898.0,CRED,Continuador,F,3m,3.0,5.880,58.5,40,PTIN,Recup,NaN,SI,SI,SI,Mamá,2
2,2019-07-09,84898.0,CRED,Continuador,F,4m,4.0,6.070,59.6,--,PTIN,SI,NaN,SI,SI,SI,Mamá,2
3,2019-08-07,84898.0,CRED,Continuador,F,5m,5.0,6.945,60.8,42.5,PTIN,SI,NaN,SI,SI,SI,Mamá,2
4,2019-09-09,84898.0,CRED,Continuador,F,6m,6.0,7.705,63.1,43.4,PTIN,SI,NaN,SI,SI,SI,Mamá,2


In [16]:
df_desnutricion_old.head()

,Fecha,Nombre_Paciente,Nº_HC,Tipo_Paciente,Categoría,Sexo,Edad,Nº_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera
0,2014-10-01,Abad Espinoza Lyam,57921.0,CRED,N,M,1m,1,4.180,53.4,37,Normal,+-1,+-1,-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3
1,2014-11-06,Abad Espinoza Lyam,57921.0,CRED,C,M,2m,2,5.530,57.6,38,Normal,+-1,+-1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,8
2,2014-12-03,Abad Espinoza Lyam,57921.0,CRED,C,M,3m,3,6.320,61.2,39,Normal,+1,+1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2
3,2015-01-05,Abad Espinoza Lyam,57921.0,CRED,C,M,4m,4,7.090,64,39.5,Normal,+-1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2
4,2015-02-02,Abad Espinoza Lyam,57921.0,CRED,C,M,5m,5,7.610,66.1,NaN,Normal,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3


In [17]:
df_desarrollo_old.head()

,Fecha,Nombre_Paciente,Nº_HC,Tipo_Paciente,Categoría,Sexo,Edad,Nº_Control,Peso,Talla,CabPC,Diag_Nacimiento,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc,Acompaña_control,Enfermera
0,2016-01-05,Abad Espinoza Lyam,57921.0,CRED,R Año Anter,M,1a4m,4,9.330,77.5,NaN,Normal,SI,NaN,SI,SI,EP,Mamá,7
1,2016-03-01,Abad Espinoza Lyam,57921.0,CRED,C,M,1a6m,5,9.610,79.5,NaN,Normal,SI,NaN,SI,SI,SI,Mamá,6
2,2016-12-20,Abad Espinoza Lyam,57921.0,CRED,C,M,2a4m,1,12.650,87.5,NaN,Normal,SI,NaN,SI,EP,EP,Mamá,3
3,2015-01-05,Abad Espinoza Lyam,57921.0,CRED,C,M,4m,4,7.090,64,39.5,Normal,SI,NaN,SI,SI,SI,Mamá,2
4,2015-02-02,Abad Espinoza Lyam,57921.0,CRED,C,M,5m,5,7.610,66.1,NaN,Normal,SI,NaN,SI,SI,SI,Mamá,3


### Tratamiento inicial para consolidacon de datasets

In [18]:
print("Shape df_desnutricion:", df_desnutricion.shape)
print("Shape df_desarrollo:", df_desarrollo.shape)

print("Shape df_desnutricion_old:", df_desnutricion_old.shape)
print("Shape df_desarrollo_old:", df_desarrollo_old.shape)

Shape df_desnutricion: (257178, 27)
Shape df_desarrollo: (257178, 18)
Shape df_desnutricion_old: (197832, 28)
Shape df_desarrollo_old: (197832, 19)


In [19]:
df_desnutricion.rename(columns={'Nº_HC': 'N_HC'}, inplace=True)
df_desnutricion.rename(columns={'Nº_Control': 'N_Control'}, inplace=True)
df_desnutricion_old.rename(columns={'Nº_HC': 'N_HC'}, inplace=True)
df_desnutricion_old.rename(columns={'Nº_Control': 'N_Control'}, inplace=True)

df_desarrollo.rename(columns={'Nº_HC': 'N_HC'}, inplace=True)
df_desarrollo.rename(columns={'Nº_Control': 'N_Control'}, inplace=True)
df_desarrollo_old.rename(columns={'Nº_HC': 'N_HC'}, inplace=True)
df_desarrollo_old.rename(columns={'Nº_Control': 'N_Control'}, inplace=True)

In [20]:
# Duplicados exactos
print("Duplicados en df_desnutricion:", df_desnutricion.duplicated().sum())
print("Duplicados en df_desarrollo:", df_desarrollo.duplicated().sum())

# Duplicados exactos
print("Duplicados en df_desnutricion:", df_desnutricion_old.duplicated().sum())
print("Duplicados en df_desarrollo:", df_desarrollo_old.duplicated().sum())

Duplicados en df_desnutricion: 12
Duplicados en df_desarrollo: 13
Duplicados en df_desnutricion: 14
Duplicados en df_desarrollo: 17


In [21]:
df_desarrollo[df_desarrollo.duplicated(keep=False)]

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc,Acompaña_control,Enfermera
40428,2024-03-27,89854.0,CRED,Continuador,F,8m,7.0,8.940,68.5,43,Normal,EP,SI,SI,SI,SI,Mamá,2
40429,2024-03-27,89854.0,CRED,Continuador,F,8m,7.0,8.940,68.5,43,Normal,EP,SI,SI,SI,SI,Mamá,2
66811,2017-07-03,68053.0,CRED,Continuador,F,8m,8.0,8.950,69.5,NaN,PTIN,SI,NaN,SI,SI,SI,Mamá,5
66812,2017-07-03,68053.0,CRED,Continuador,F,8m,8.0,8.950,69.5,NaN,PTIN,SI,NaN,SI,SI,SI,Mamá,5
68512,2022-11-08,87289.0,Control RN,Continuador,F,22d,2.0,4.700,54.5,38,Normal,NaN,NaN,NaN,NaN,NaN,Ambos,0
68513,2022-11-08,87289.0,Control RN,Continuador,F,22d,2.0,4.700,54.5,38,Normal,NaN,NaN,NaN,NaN,NaN,Ambos,0
103409,2024-02-07,91801.0,Control RN,Continuador,F,22d,2.0,3.900,51.8,35.2,Normal,NaN,NaN,NaN,NaN,NaN,Mamá,0
103410,2024-02-07,91801.0,Control RN,Continuador,F,22d,2.0,3.900,51.8,35.2,Normal,NaN,NaN,NaN,NaN,NaN,Mamá,0
119236,2024-03-27,89714.0,CRED,Continuador,M,8m,8.0,8.730,69.9,45.5,Normal,EP,SI,SI,SI,SI,Mamá,2
119237,2024-03-27,89714.0,CRED,Continuador,M,8m,8.0,8.730,69.9,45.5,Normal,EP,SI,SI,SI,SI,Mamá,2


In [22]:
# Se eliminan registros duplicados exactos de ambos datasets y se resetea el index
df_desnutricion = df_desnutricion.drop_duplicates().reset_index(drop=True)
df_desarrollo   = df_desarrollo.drop_duplicates().reset_index(drop=True)

df_desnutricion_old = df_desnutricion_old.drop_duplicates().reset_index(drop=True)
df_desarrollo_old   = df_desarrollo_old.drop_duplicates().reset_index(drop=True)

print("Duplicados en df_desnutricion:", df_desnutricion.duplicated().sum())
print("Duplicados en df_desarrollo:", df_desarrollo.duplicated().sum())

print("Duplicados en df_desnutricion2:", df_desnutricion_old.duplicated().sum())
print("Duplicados en df_desarrollo2:", df_desarrollo_old.duplicated().sum())

Duplicados en df_desnutricion: 0
Duplicados en df_desarrollo: 0
Duplicados en df_desnutricion2: 0
Duplicados en df_desarrollo2: 0


### Corrigiendo NHC del dataset 2009-2016

TANI nos mencionó que habian pacientes en el dataset pre pandemia que no tenian NHC y que se les habia asignado por 0 o estaban registrados como nulos. Se procede a asignar un NHC unico a cada paciente que no lo tiene, empezando desde 15000

In [26]:
## CORRECCION EN DATASET OLD
def asignar_codigos_nhc(df, columna_nombre='Nombre_Paciente', columna_nhc='N_HC', codigo_inicial=13000):
    """
    Asigna códigos N_HC únicos a pacientes agrupados por nombre
    
    Parámetros:
    - df: DataFrame con los datos
    - columna_nombre: nombre de la columna que contiene los nombres
    - columna_nhc: nombre de la columna N_HC
    - codigo_inicial: código inicial para asignar (default 15000)
    """
    
    # Hacer una copia para no modificar el original
    df_copy = df.copy()
    
    # Filtrar registros con N_HC nulo O igual a 0, y que tengan nombre válido
    sin_codigo = df_copy[
        (df_copy[columna_nhc].isnull() | (df_copy[columna_nhc] == 0)) & 
        (df_copy[columna_nombre].notna()) & 
        (df_copy[columna_nombre] != '')
    ]
    
    print(f"Registros sin código N_HC (nulos o ceros) pero con nombre: {len(sin_codigo):,}")
    
    # Mostrar desglose
    nulos = df_copy[df_copy[columna_nhc].isnull() & df_copy[columna_nombre].notna() & (df_copy[columna_nombre] != '')]
    ceros = df_copy[(df_copy[columna_nhc] == 0) & df_copy[columna_nombre].notna() & (df_copy[columna_nombre] != '')]
    
    print(f"  - Registros con N_HC nulo: {len(nulos):,}")
    print(f"  - Registros con N_HC = 0: {len(ceros):,}")
    
    if len(sin_codigo) == 0:
        print("No hay registros para procesar")
        return df_copy
    
    # Obtener nombres únicos
    nombres_unicos = sin_codigo[columna_nombre].unique()
    print(f"Pacientes únicos sin código: {len(nombres_unicos):,}")
    
    # Crear mapeo de nombre a nuevo código
    mapeo_codigos = {}
    codigo_actual = codigo_inicial
    
    for nombre in nombres_unicos:
        mapeo_codigos[nombre] = codigo_actual
        codigo_actual += 1
    
    # Aplicar los nuevos códigos (tanto para nulos como para ceros)
    for nombre, nuevo_codigo in mapeo_codigos.items():
        mask = (df_copy[columna_nhc].isnull() | (df_copy[columna_nhc] == 0)) & (df_copy[columna_nombre] == nombre)
        df_copy.loc[mask, columna_nhc] = nuevo_codigo
    
    # Mostrar resumen
    print(f"\n=== RESUMEN DE ASIGNACIÓN ===")
    print(f"Códigos asignados: {codigo_inicial} - {codigo_actual-1}")
    print(f"Total de pacientes procesados: {len(nombres_unicos):,}")
    print(f"Registros actualizados: {len(sin_codigo):,}")
    
    # Verificación final
    restantes_nulos = df_copy[columna_nhc].isnull().sum()
    restantes_ceros = (df_copy[columna_nhc] == 0).sum()
    print(f"N_HC nulos restantes: {restantes_nulos:,}")
    print(f"N_HC = 0 restantes: {restantes_ceros:,}")
    
    return df_copy, mapeo_codigos

In [27]:
df_desnutricion_old[df_desnutricion_old['N_HC'] == 0]['Nombre_Paciente'].value_counts()

Nombre_Paciente
Ojeda Pari Alejandra                   11
Fernandez Reyes Angel Misael           10
Robles Cama Maybet                     10
Rodríguez Calle Enma                    8
Tello Atoco Diego                       6
Acuña Cordova Kimberly                  5
Chacon Tasaico Alizee                   4
Silva Quinta Tracy                      4
Idelfonso Soto Maria                    4
Regino Casas Maryorie                   4
Alvarez Martinez Ezequiel               3
Quispe Roque Lucia                      3
Padilla León Elias                      3
Huamancayo Carril Maria Liz             3
Huaman Herrera Matias                   3
Rojas Carrizales Isai                   3
Sanchez Garay Luis Rodrigo              3
Orrego Camposano Luis Fernando          2
Quispe Perez Manuel                     2
Valdivia Castillo Cristina              2
Vergara Cervantes Luis                  2
Pituy Pinedo Celia                      2
Lopez Huaman Eileen                     1
Farfan Huaman Xiom

In [28]:
df_desarrollo_old[df_desarrollo_old['N_HC'] == 0]['Nombre_Paciente'].value_counts()

Nombre_Paciente
Fernandez Reyes Angel Misael           10
Acuña Cordova Kimberly                  5
Idelfonso Soto Maria                    4
Chacon Tasaico Alizee                   4
Huamancayo Carril Maria Liz             3
Alvarez Martinez Ezequiel               3
Huaman Herrera Matias                   3
Valdivia Castillo Cristina              2
Farfan Huaman Xiomara K                 1
Fernandez Dueñas Dafne                  1
Lopez Huaman Eileen                     1
Sarmientos Mosquera Alberto Enrique     1
Name: count, dtype: int64

In [29]:
# Se escoge el dataset de nutricion porque tiene mas registros
df_desnutricion_old, mapeo = asignar_codigos_nhc(df_desnutricion_old, 'Nombre_Paciente', 'N_HC', 15000)

Registros sin código N_HC (nulos o ceros) pero con nombre: 133
  - Registros con N_HC nulo: 23
  - Registros con N_HC = 0: 110
Pacientes únicos sin código: 43

=== RESUMEN DE ASIGNACIÓN ===
Códigos asignados: 15000 - 15042
Total de pacientes procesados: 43
Registros actualizados: 133
N_HC nulos restantes: 0
N_HC = 0 restantes: 0


In [30]:
df_desnutricion_old.head()

,Fecha,Nombre_Paciente,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera
0,2014-10-01,Abad Espinoza Lyam,57921.0,CRED,N,M,1m,1,4.180,53.4,37,Normal,+-1,+-1,-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3
1,2014-11-06,Abad Espinoza Lyam,57921.0,CRED,C,M,2m,2,5.530,57.6,38,Normal,+-1,+-1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,8
2,2014-12-03,Abad Espinoza Lyam,57921.0,CRED,C,M,3m,3,6.320,61.2,39,Normal,+1,+1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2
3,2015-01-05,Abad Espinoza Lyam,57921.0,CRED,C,M,4m,4,7.090,64,39.5,Normal,+-1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2
4,2015-02-02,Abad Espinoza Lyam,57921.0,CRED,C,M,5m,5,7.610,66.1,NaN,Normal,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3


Deteccion de casos de duplicados con variaciones en registros. Ejemplo: Mismo paciente y con misma fecha pero con diferente registro de peso

In [34]:
print("DATASET ANTIGUO")
# DUPLICADOS POR LLAVES (no solo registros exactos)
keys = ['Fecha', 'Nombre_Paciente', 'Tipo_Paciente','N_Control']

print("Registros df_desnutricion:", df_desnutricion_old.shape)
print("Registros df_desarrollo:",   df_desarrollo_old.shape)

# En df_desnutricion
dups_desnutricion_old = df_desnutricion_old[df_desnutricion_old.duplicated(subset=keys, keep=False)]
print(f"Duplicados en df_desnutricion: {dups_desnutricion_old.shape[0]}")

# En df_desarrollo
dups_desarrollo_old = df_desarrollo_old[df_desarrollo_old.duplicated(subset=keys, keep=False)]
print(f"Duplicados en df_desarrollo: {dups_desarrollo_old.shape[0]}")

DATASET ANTIGUO
Registros df_desnutricion: (197818, 28)
Registros df_desarrollo: (197815, 19)
Duplicados en df_desnutricion: 50
Duplicados en df_desarrollo: 44


In [35]:
print("DATASET MAS RECIENTE")
# DUPLICADOS POR LLAVES (no solo registros exactos)
keys = ['Fecha', 'N_HC', 'Tipo_Paciente','N_Control']

print("Registros df_desnutricion:", df_desnutricion.shape)
print("Registros df_desarrollo:",   df_desarrollo.shape)

# En df_desnutricion
dups_desnutricion = df_desnutricion[df_desnutricion.duplicated(subset=keys, keep=False)]
print(f"Duplicados en df_desnutricion: {dups_desnutricion.shape[0]}")

# En df_desarrollo
dups_desarrollo = df_desarrollo[df_desarrollo.duplicated(subset=keys, keep=False)]
print(f"Duplicados en df_desarrollo: {dups_desarrollo.shape[0]}")

DATASET MAS RECIENTE
Registros df_desnutricion: (257166, 27)
Registros df_desarrollo: (257165, 18)
Duplicados en df_desnutricion: 32
Duplicados en df_desarrollo: 30


In [37]:
dups_desnutricion_old.sort_values('N_HC').head(8)

,Fecha,Nombre_Paciente,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera
83972,2015-01-17,Huamani Arana María de los Angeles,42492.0,CRED,C,F,4a,1,15.750,98.2,NaN,PTIN,+1,<-1,-1,GIP-GIT,Riesgo T/E,CA,Desf,NaN,NaN,NaN,SI,NaN,NaN,NaN,Mamá,2
83973,2015-01-17,Huamani Arana María de los Angeles,42492.0,CRED,C,F,4a,1,15.750,98.2,NaN,PTIN,+1,<-1,-1,GIP-GIT,Riesgo T/E,CA,Fav,NaN,NaN,NaN,SI,NaN,NaN,NaN,Mamá,2
184668,2011-12-09,Valle Medina Dayiro Jhanyol,44027.0,CRED,C,M,7m,7,8.220,67,NaN,Prétermino,+1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,NaN,NaN,Si,NaN,NaN,NaN,Mamá,1
184667,2011-12-09,Valle Medina Dayiro Jhanyol,44027.0,CRED,C,M,7m,7,7.000,64,NaN,Prétermino,+1,<-1,-1,GAP-GAT,Riesgo T/E,CA,Fav,NaN,NaN,NaN,Si,NaN,NaN,NaN,Mamá,1
184671,2012-02-07,Valle Medina Dayiro Jhanyol,44027.0,CRED,C,M,9m,9,8.700,70.5,NaN,Prétermino,+1,+-1,+1,GIP-GIT,Normal,NE,Fav,NaN,NaN,NaN,Si,NaN,NaN,NaN,Mamá,1
184670,2012-02-07,Valle Medina Dayiro Jhanyol,44027.0,CRED,C,M,9m,9,7.600,67.3,NaN,Prétermino,-1,<-1,-1,GAP-GIT,Riesgo T/E,CA,Fav,NaN,NaN,NaN,Si,NaN,NaN,NaN,Mamá,1
147475,2012-01-17,Ramos Herrera Jorge Josue,45259.0,CRED,C,M,4m,5,6.350,61.6,40.3,BPN,-1,-1,-1,GAP-GIT,Riesgo P/E,CA,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,6
147474,2012-01-17,Ramos Herrera Jorge Josue,45259.0,CRED,C,M,4m,5,6.370,62.3,41.3,BPN,-1,-1,-1,GAP-GAT,Riesgo P/E,CA,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,6


In [38]:
dups_desnutricion.sort_values('N_HC').head(8)

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera
140019,2017-02-24,60235.0,CRED,Continuador,F,2a,1.0,12.320,81.2,NaN,Normal,>+1,-2,+1,GAP-GIT,Riesgo T/E,CA,Desf,NaN,NaN,NaN,SI,NaN,NaN,NaN,Mamá,2
148127,2017-02-24,60235.0,CRED,Continuador,F,2a,1.0,13.440,83.6,NaN,Normal,>+2,<-1,+1,GIP-GIT,Riesgo T/E,CN,Desf,NaN,NaN,NaN,NO,NaN,NaN,NaN,Mamá,2
181650,2017-09-13,63828.0,CRED,Continuador,M,2a,1.0,12.400,81.4,NaN,Prétermino,>+1,<-1,+1,GIP-GIT,Riesgo T/E,CA,Desf,NaN,NaN,NaN,NO,NaN,NaN,NaN,Mamá,2
181651,2017-09-13,63828.0,CRED,Continuador,M,2a,1.0,12.600,81.9,NaN,Prétermino,>+1,<-1,+1,GAP-GIT,Riesgo T/E,CA,Desf,NaN,NaN,NaN,SI,NaN,NaN,NaN,Mamá,2
196927,2017-09-25,67002.0,CRED,Continuador,F,1a4m,3.0,10.120,77.7,NaN,Prétermino,+1,-1,-1,GAP-GAT,Riesgo P/E,CA,Fav,NaN,NaN,NaN,NO,NaN,NaN,NaN,Mamá,7
196926,2017-09-25,67002.0,CRED,Continuador,F,1a4m,3.0,10.340,77.1,NaN,Prétermino,+1,-1,+-1,GAP-GAT,Normal,NaN,NaN,NaN,NaN,NaN,NO,NaN,NaN,NaN,Mamá,7
189640,2017-04-03,68614.0,CRED,Continuador,M,3m,3.0,7.140,61.6,42,Normal,>+1,+1,+1,GAP-GIT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2
189639,2017-04-03,68614.0,CRED,Continuador,M,3m,3.0,6.025,58.4,39.5,Normal,+1,<-1,-1,GIP-GIT,Riesgo T/E,CN,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2


In [39]:
dups_desarrollo_old.sort_values('N_HC').head(8)

,Fecha,Nombre_Paciente,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc,Acompaña_control,Enfermera
184671,2011-12-09,Valle Medina Dayiro Jhanyol,44027.0,CRED,C,M,7m,7,7.000,64,NaN,Prétermino,Recup,NaN,SI,SI,SI,Mamá,1
184672,2011-12-09,Valle Medina Dayiro Jhanyol,44027.0,CRED,C,M,7m,7,8.220,67,NaN,Prétermino,Recup,NaN,SI,SI,SI,Mamá,1
184660,2012-02-07,Valle Medina Dayiro Jhanyol,44027.0,CRED,C,M,9m,9,7.600,67.3,NaN,Prétermino,SI,NaN,SI,SI,SI,Mamá,1
184659,2012-02-07,Valle Medina Dayiro Jhanyol,44027.0,CRED,C,M,9m,9,8.700,70.5,NaN,Prétermino,Recup,NaN,SI,SI,SI,Mamá,1
147486,2012-01-17,Ramos Herrera Jorge Josue,45259.0,CRED,C,M,4m,5,6.370,62.3,41.3,BPN,SI,NaN,SI,SI,SI,Mamá,6
147487,2012-01-17,Ramos Herrera Jorge Josue,45259.0,CRED,C,M,4m,5,6.350,61.6,40.3,BPN,SI,NaN,SI,SI,SI,Mamá,6
183502,2015-06-20,Valdivia Diaz Matias,46925.0,CRED,C,M,3a4m,3,16.700,100.4,NaN,PTIN,SI,NaN,SI,SI,SI,Mamá,5
183501,2015-06-20,Valdivia Diaz Matias,46925.0,CRED,C,M,3a4m,3,16.700,100.4,NaN,PTIN,SI,NaN,SI,SI,SI,Mamá,7


In [40]:
dups_desarrollo.sort_values('N_HC').head(8)

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc,Acompaña_control,Enfermera
140019,2017-02-24,60235.0,CRED,Continuador,F,2a,1.0,12.320,81.2,NaN,Normal,SI,NaN,SI,SI,SI,Mamá,2
148127,2017-02-24,60235.0,CRED,Continuador,F,2a,1.0,13.440,83.6,NaN,Normal,SI,NaN,SI,SI,SI,Mamá,2
181650,2017-09-13,63828.0,CRED,Continuador,M,2a,1.0,12.400,81.4,NaN,Prétermino,SI,NaN,SI,SI,SI,Mamá,2
181651,2017-09-13,63828.0,CRED,Continuador,M,2a,1.0,12.600,81.9,NaN,Prétermino,SI,NaN,SI,SI,SI,Mamá,2
196927,2017-09-25,67002.0,CRED,Continuador,F,1a4m,3.0,10.120,77.7,NaN,Prétermino,Recup,NaN,SI,SI,SI,Mamá,7
196926,2017-09-25,67002.0,CRED,Continuador,F,1a4m,3.0,10.340,77.1,NaN,Prétermino,EP,NaN,SI,SI,SI,Mamá,7
189640,2017-04-03,68614.0,CRED,Continuador,M,3m,3.0,7.140,61.6,42,Normal,SI,NaN,SI,SI,SI,Mamá,2
189639,2017-04-03,68614.0,CRED,Continuador,M,3m,3.0,6.025,58.4,39.5,Normal,EP,NaN,SI,SI,SI,Mamá,2


In [41]:
## SE GUARDARAN LOS CASOS ENCONTRADOS PARA VALIDACION CON LA ORGANIZACION
dups_desnutricion_old.to_csv('../data/intermediate/TANI_nutricion_casos_duplicados_base_pre_2017.csv',sep=";")
dups_desarrollo_old.to_csv('../data/intermediate/TANI_desarrollo_casos_duplicados_base_pre_2017.csv',sep=";")
dups_desnutricion.to_csv('../data/intermediate/TANI_nutricion_casos_duplicados_base_actual_2017_2025.csv',sep=";")
dups_desarrollo.to_csv('../data/intermediate/TANI_desarrollo_casos_duplicados_base_actual_2017_2025.csv',sep=";")

Se observan registros duplicados con la principal diferencia en los campos numericos (peso, talla, etc). Lo que evidencia un posible mal registro de datos en la digitalizacion de las HC.

In [42]:
keys = ['Fecha', 'Nombre_Paciente', 'Tipo_Paciente','N_Control']

mask_dup_desnut_old = df_desnutricion_old.duplicated(subset=keys, keep=False)
mask_dup_desarr_old = df_desarrollo_old.duplicated(subset=keys, keep=False)

df_desnutricion_uniques_old = df_desnutricion_old[~mask_dup_desnut_old].copy()
df_desarrollo_uniques_old   = df_desarrollo_old[~mask_dup_desarr_old].copy()

print("Duplicados en df_desnutricion:", int(mask_dup_desnut_old.sum()))
print("Duplicados en df_desarrollo:",   int(mask_dup_desarr_old.sum()))

print("Filas únicas df_desnutricion:", df_desnutricion_uniques_old.shape)
print("Filas únicas df_desarrollo:",   df_desarrollo_uniques_old.shape)


Duplicados en df_desnutricion: 50
Duplicados en df_desarrollo: 44
Filas únicas df_desnutricion: (197768, 28)
Filas únicas df_desarrollo: (197771, 19)


In [43]:
keys = ['Fecha', 'N_HC', 'Tipo_Paciente','N_Control']

mask_dup_desnut = df_desnutricion.duplicated(subset=keys, keep=False)
mask_dup_desarr = df_desarrollo.duplicated(subset=keys, keep=False)

df_desnutricion_uniques = df_desnutricion[~mask_dup_desnut].copy()
df_desarrollo_uniques   = df_desarrollo[~mask_dup_desarr].copy()

print("Duplicados en df_desnutricion:", int(mask_dup_desnut.sum()))
print("Duplicados en df_desarrollo:",   int(mask_dup_desarr.sum()))

print("Filas únicas df_desnutricion:", df_desnutricion_uniques.shape)
print("Filas únicas df_desarrollo:",   df_desarrollo_uniques.shape)


Duplicados en df_desnutricion: 32
Duplicados en df_desarrollo: 30
Filas únicas df_desnutricion: (257134, 27)
Filas únicas df_desarrollo: (257135, 18)


Se separan los casos encontrados antes de realizar el merge entre bases

In [44]:
cols_desarrollo_old = ['Fecha', 'Nombre_Paciente', 'N_Control','Tipo_Paciente', '(M) - FG', '(M) - FF', '(C) - Cog','(L) - Len','(S) - Soc']
df_desarrollo_reducido_uniques_old = df_desarrollo_uniques_old[cols_desarrollo_old]

In [ ]:
# Merge datasets pre 2017
df_consolidado_old = df_desnutricion_uniques_old.merge(
    df_desarrollo_reducido_uniques_old,
    how='inner',
    on=['Fecha', 'Nombre_Paciente', 'Tipo_Paciente','N_Control']
)

In [46]:
df_consolidado_old.head()

,Fecha,Nombre_Paciente,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc
0,2014-10-01,Abad Espinoza Lyam,57921.0,CRED,N,M,1m,1,4.180,53.4,37,Normal,+-1,+-1,-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,EP,NaN,SI,SI,SI
1,2014-11-06,Abad Espinoza Lyam,57921.0,CRED,C,M,2m,2,5.530,57.6,38,Normal,+-1,+-1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,8,EP,NaN,SI,SI,SI
2,2014-12-03,Abad Espinoza Lyam,57921.0,CRED,C,M,3m,3,6.320,61.2,39,Normal,+1,+1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,Recup,NaN,SI,SI,SI
3,2015-01-05,Abad Espinoza Lyam,57921.0,CRED,C,M,4m,4,7.090,64,39.5,Normal,+-1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI
4,2015-02-02,Abad Espinoza Lyam,57921.0,CRED,C,M,5m,5,7.610,66.1,NaN,Normal,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,SI,NaN,SI,SI,SI


In [47]:
df_consolidado_old.shape

(197768, 33)

In [48]:
cols_desarrollo = ['Fecha', 'N_HC', 'N_Control','Tipo_Paciente', '(M) - FG', '(M) - FF', '(C) - Cog','(L) - Len','(S) - Soc']
df_desarrollo_reducido_uniques = df_desarrollo_uniques[cols_desarrollo]

In [49]:
## MERGE DATASET RECIENTE
df_consolidado = df_desnutricion_uniques.merge(
    df_desarrollo_reducido_uniques,
    how='inner',  # o 'inner' si quieres solo los que tienen match en ambos
    on=['Fecha', 'N_HC', 'Tipo_Paciente','N_Control']
)

In [50]:
df_consolidado.head()

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc
0,2019-05-09,84898.0,CRED,Continuador,F,2m,2.0,4.945,55,38.4,PTIN,+1,<-1,-1,GIP-GIT,Riesgo T/E,CN,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Papá,4,EP,NaN,SI,SI,SI
1,2019-06-07,84898.0,CRED,Continuador,F,3m,3.0,5.880,58.5,40,PTIN,+1,<-1,-1,GIP-GIT,Riesgo T/E,CA,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,Recup,NaN,SI,SI,SI
2,2019-07-09,84898.0,CRED,Continuador,F,4m,4.0,6.070,59.6,--,PTIN,+1,-2,<-1,GIP-GIT,Riesgo P/E,CA,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI
3,2019-08-07,84898.0,CRED,Continuador,F,5m,5.0,6.945,60.8,42.5,PTIN,>+1,<-2,-1,GAP-GIT,Riesgo T/E,CA,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI
4,2019-09-09,84898.0,CRED,Continuador,F,6m,6.0,7.705,63.1,43.4,PTIN,>+1,-2,-1,GAP-GIT,Riesgo T/E,CA,Desf,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI


In [51]:
df_consolidado.shape

(257133, 32)

In [52]:
df_consolidado_old.sort_values('N_HC')

,Fecha,Nombre_Paciente,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc
178771,2011-01-12,Tomaylla Denegri Leonel,3340.0,CRED,C,M,1a11m,3,12.000,84.6,NaN,Normal,+1,-1,+-1,GIP-GIT,Normal,NE,Fav,NaN,NaN,NaN,No,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,EP
178770,2010-10-14,Tomaylla Denegri Leonel,3340.0,CRED,C,M,1a9m,2,11.200,83,NaN,Normal,+1,-1,-1,GIP-GIT,Normal,NE,Fav,NaN,NaN,NaN,No,NaN,NaN,NaN,Mamá,6,SI,NaN,SI,SI,EP
174985,2010-06-07,Taparaco Pocco Maycol,3780.0,CRED,C,M,5m,4,6.650,65,42,Normal,-1,-1,-1,GIP-GAT,Riesgo P/E,CN,Fav,D. Aguda a Riesgo,LMX,4,NaN,NaN,NaN,NaN,Mamá,1,SI,NaN,SI,SI,SI
174986,2010-07-07,Taparaco Pocco Maycol,3780.0,CRED,C,M,6m,5,7.300,69,43,Normal,<-1,+1,-1,GAP-GAT,Riesgo P/E,CA,Fav,NaN,LMX,4,NaN,NaN,NaN,NaN,Mamá,5,SI,NaN,SI,SI,SI
174989,2010-10-11,Taparaco Pocco Maycol,3780.0,CRED,C,M,9m,8,8.500,73,NaN,Normal,-1,+1,-1,GAP-GAT,Riesgo P/T,CA,Fav,NaN,NaN,NaN,Si,NaN,NaN,NaN,Mamá,6,SI,NaN,SI,SI,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20706,2013-06-10,Benavente Pinares Dann,83048.0,CRED,C,M,9m,7,10.920,75,NaN,Normal,>+1,>+1,>+1,GAP-GAT,Normal,NE,Fav,NaN,NaN,NaN,No,NaN,NaN,NaN,Mamá,4,SI,NaN,SI,SI,SI
20705,2013-04-01,Benavente Pinares Dann,83048.0,CRED,C,M,7m,6,10.050,71,NaN,Normal,>+2,+1,>+2,GAP-GAT,Normal,NE,Fav,NaN,NaN,NaN,Si,NaN,NaN,NaN,Mamá,7,SI,NaN,SI,SI,SI
20704,2013-03-01,Benavente Pinares Dann,83048.0,CRED,C,M,6m,5,9.650,69.5,NaN,Normal,>+1,+1,+1,GAP-GAT,Normal,NE,Norm,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,7,SI,NaN,SI,SI,SI
20708,2013-09-06,Benavente Pinares Dann,83048.0,CRED,C,M,1a,1,11.470,77.5,NaN,Normal,>+1,+1,>+1,GIP-GAT,Normal,NE,Fav,NaN,NaN,NaN,Si,NaN,NaN,NaN,Mamá,7,SI,NaN,SI,SI,SI


In [53]:
df_consolidado_old.columns

Index(['Fecha', 'Nombre_Paciente', 'N_HC', 'Tipo_Paciente', 'Categoría',
       'Sexo', 'Edad', 'N_Control', 'Peso', 'Talla', 'CabPC',
       'Diag_Nacimiento', 'P/T', 'T/E', 'P/E', 'Ganancia_Peso_Talla',
       'Dx_Nutricional', 'CN-CA', 'Mantiene_Diag_Fav/Desf', 'Recuperado',
       'Lactancia', 'Razón', 'ACA', 'Tam_para', 'Tam_graha', 'Tam_hb',
       'Acompaña_control', 'Enfermera', '(M) - FG', '(M) - FF', '(C) - Cog',
       '(L) - Len', '(S) - Soc'],
      dtype='object')

In [55]:
df_consolidado_nhc_nulos = df_consolidado[df_consolidado['N_HC'].isnull()]
df_consolidado_nhc_nulos

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc
84530,2017-10-12,NaN,CRED,Nuevo,F,29d,1.0,3.660,50.7,35.9,Prétermino,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,6,EP,NaN,SI,SI,SI
84531,2017-11-13,NaN,CRED,Continuador,F,2m,2.0,4.925,55.6,38.7,Prétermino,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,6,EP,NaN,SI,SI,SI
84532,2017-12-12,NaN,CRED,Continuador,F,3m,3.0,5.635,58.5,39.8,Prétermino,+1,+1,+1,GIP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,6,EP,NaN,SI,SI,SI
84533,2018-01-11,NaN,CRED,Continuador,F,4m,4.0,6.250,61,41,Prétermino,+1,+1,-1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,6,Recup,NaN,SI,SI,SI
84534,2018-02-12,NaN,CRED,Continuador,F,5m,5.0,6.980,63.8,42.2,Prétermino,+1,+-1,-1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,6,SI,NaN,SI,SI,SI
84535,2018-04-11,NaN,CRED,Continuador,F,7m,6.0,7.930,65.7,43.9,Prétermino,+1,-1,-1,GAP-GAT,Normal,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN,Mamá,6,SI,NaN,SI,SI,SI
84536,2018-05-10,NaN,CRED,Continuador,F,8m,7.0,8.235,67.5,44.8,Prétermino,+1,-1,+-1,GAP-GAT,Normal,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN,Mamá,6,SI,NaN,SI,SI,SI
84537,2018-06-12,NaN,CRED,Continuador,F,9m,8.0,8.520,68.7,45.5,Prétermino,+1,-1,-1,GIP-GAT,Normal,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,NaN,Mamá,6,SI,NaN,SI,SI,SI
84538,2018-08-10,NaN,CRED,Continuador,F,11m,9.0,9.045,70.3,46.2,Prétermino,+1,<-1,-1,GIP-GIT,Riesgo T/E,CN,Desf,NaN,NaN,NaN,SI,NaN,NaN,NaN,Mamá,6,SI,NaN,SI,SI,SI


In [56]:
df_consolidado[df_consolidado['N_HC']==14999]

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc


In [57]:
# Imputacion del caso con N_HC nulo (SOLO UNO EN EL DATASET RECIENTE SIN NOMBRES )
df_consolidado['N_HC'] = df_consolidado['N_HC'].fillna(14999)

In [59]:
df_consolidado.columns

Index(['Fecha', 'N_HC', 'Tipo_Paciente', 'Categoría', 'Sexo', 'Edad',
       'N_Control', 'Peso', 'Talla', 'CabPC', 'Diag_Nacimiento', 'P/T', 'T/E',
       'P/E', 'Ganancia_Peso_Talla', 'Dx_Nutricional', 'CN-CA',
       'Mantiene_Diag_Fav/Desf', 'Recuperado', 'Lactancia', 'Razón', 'ACA',
       'Tam_para', 'Tam_graha', 'Tam_hb', 'Acompaña_control', 'Enfermera',
       '(M) - FG', '(M) - FF', '(C) - Cog', '(L) - Len', '(S) - Soc'],
      dtype='object')

In [60]:
vars_finales=df_consolidado.columns.tolist()

### Concatenacion de datasets pre pandemia y actual

In [62]:
def concatenar_dataframes(df1, df2, resetear_index=True):
    """
    Concatena dos dataframes verticalmente
    """

    print("=== INFORMACIÓN ANTES DE CONCATENAR ===")
    print(f"DataFrame 1: {df1.shape[0]:,} filas x {df1.shape[1]} columnas")
    print(f"DataFrame 2: {df2.shape[0]:,} filas x {df2.shape[1]} columnas")

    # Verificar que tengan las mismas columnas
    cols_df1 = set(df1.columns)
    cols_df2 = set(df2.columns)

    if cols_df1 != cols_df2:
        print("\n⚠️  ADVERTENCIA: Las columnas no coinciden exactamente")
        print(f"Solo en DF1: {cols_df1 - cols_df2}")
        print(f"Solo en DF2: {cols_df2 - cols_df1}")

    # Concatenar
    df_consolidado = pd.concat([df1, df2], ignore_index=resetear_index)

    print(f"\n=== RESULTADO ===")
    print(f"DataFrame consolidado: {df_consolidado.shape[0]:,} filas x {df_consolidado.shape[1]} columnas")

    return df_consolidado

In [63]:
df_consolidado_vf = concatenar_dataframes(df_consolidado_old[vars_finales], df_consolidado)

=== INFORMACIÓN ANTES DE CONCATENAR ===
DataFrame 1: 197,768 filas x 32 columnas
DataFrame 2: 257,133 filas x 32 columnas

=== RESULTADO ===
DataFrame consolidado: 454,901 filas x 32 columnas


Se guarda el resultado de la consolidación (primer preprocessing)

In [64]:
df_consolidado_vf.to_csv("../data/intermediate/tani_consolidado_v1.csv", index=False)

In [65]:
df_consolidado_vf.head()

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc
0,2014-10-01,57921.0,CRED,N,M,1m,1.0,4.180,53.4,37,Normal,+-1,+-1,-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,EP,NaN,SI,SI,SI
1,2014-11-06,57921.0,CRED,C,M,2m,2.0,5.530,57.6,38,Normal,+-1,+-1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,8,EP,NaN,SI,SI,SI
2,2014-12-03,57921.0,CRED,C,M,3m,3.0,6.320,61.2,39,Normal,+1,+1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,Recup,NaN,SI,SI,SI
3,2015-01-05,57921.0,CRED,C,M,4m,4.0,7.090,64,39.5,Normal,+-1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI
4,2015-02-02,57921.0,CRED,C,M,5m,5.0,7.610,66.1,NaN,Normal,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,SI,NaN,SI,SI,SI


In [66]:
#df_sample = df_consolidado_vf.sample(n=100, random_state=42)  # random_state para reproducibilidad
#df_sample.to_csv("../data/intermediate/muestra_100_registros.csv", index=False)

## TRATAMIENTO

In [95]:
df = pd.read_csv('../data/intermediate/tani_consolidado_v1.csv')

C:\Users\isma_\AppData\Local\Temp\ipykernel_18028\965930684.py:1: DtypeWarning: Columns (7,8,9,20,22,23,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/intermediate/tani_consolidado_v1.csv')


In [96]:
df.shape

(454901, 32)

In [97]:
df.head()

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc
0,2014-10-01,57921.0,CRED,N,M,1m,1.0,4.18,53.4,37,Normal,+-1,+-1,-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,EP,NaN,SI,SI,SI
1,2014-11-06,57921.0,CRED,C,M,2m,2.0,5.53,57.6,38,Normal,+-1,+-1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,8,EP,NaN,SI,SI,SI
2,2014-12-03,57921.0,CRED,C,M,3m,3.0,6.32,61.2,39,Normal,+1,+1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,Recup,NaN,SI,SI,SI
3,2015-01-05,57921.0,CRED,C,M,4m,4.0,7.09,64.0,39.5,Normal,+-1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI
4,2015-02-02,57921.0,CRED,C,M,5m,5.0,7.61,66.1,NaN,Normal,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,SI,NaN,SI,SI,SI


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454901 entries, 0 to 454900
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Fecha                   454901 non-null  object 
 1   N_HC                    454901 non-null  float64
 2   Tipo_Paciente           454901 non-null  object 
 3   Categoría               454901 non-null  object 
 4   Sexo                    454901 non-null  object 
 5   Edad                    454901 non-null  object 
 6   N_Control               454900 non-null  float64
 7   Peso                    454892 non-null  object 
 8   Talla                   454878 non-null  object 
 9   CabPC                   278593 non-null  object 
 10  Diag_Nacimiento         454901 non-null  object 
 11  P/T                     407210 non-null  object 
 12  T/E                     407216 non-null  object 
 13  P/E                     407210 non-null  object 
 14  Ganancia_Peso_Talla 

In [98]:
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

In [99]:
# Convierte a numérico columnas típicas (ignora las que no existan)
for c in ['N_HC','N_Control','Peso','Talla','CabPC']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

In [100]:
df.columns

Index(['Fecha', 'N_HC', 'Tipo_Paciente', 'Categoría', 'Sexo', 'Edad',
       'N_Control', 'Peso', 'Talla', 'CabPC', 'Diag_Nacimiento', 'P/T', 'T/E',
       'P/E', 'Ganancia_Peso_Talla', 'Dx_Nutricional', 'CN-CA',
       'Mantiene_Diag_Fav/Desf', 'Recuperado', 'Lactancia', 'Razón', 'ACA',
       'Tam_para', 'Tam_graha', 'Tam_hb', 'Acompaña_control', 'Enfermera',
       '(M) - FG', '(M) - FF', '(C) - Cog', '(L) - Len', '(S) - Soc'],
      dtype='object')

In [101]:
def edad_a_meses(s):
    """
    Convierte diferentes formatos de edad a meses.
    
    Formatos soportados:
    - "1a8m" -> 1 año 8 meses = 20 meses
    - "2a" -> 2 años = 24 meses  
    - "11m" -> 11 meses
    - "28d" -> 28 días = ~0.93 meses
    - "1a2m15d" -> 1 año 2 meses 15 días = ~14.5 meses
    - "11" (solo número) -> 11 meses
    
    Returns:
        float: Edad en meses (con decimales para días)
    """
    if pd.isna(s): 
        return np.nan
    
    s = str(s).lower().strip()
    
    # Buscar años, meses y días usando regex
    a = re.search(r'(\d+)\s*a', s)  # años
    m = re.search(r'(\d+)\s*m', s)  # meses  
    d = re.search(r'(\d+)\s*d', s)  # días
    
    # Extraer valores (0 si no se encuentra)
    anios = int(a.group(1)) if a else 0
    meses = int(m.group(1)) if m else 0
    dias = int(d.group(1)) if d else 0
    
    # Convertir todo a meses
    # 1 año = 12 meses, 1 mes ≈ 30.44 días (promedio)
    meses_totales = anios * 12 + meses + dias / 30.44
    
    return round(meses_totales, 2)

In [102]:
# Se convierte la edad categorica a numérica en meses
if 'Edad' in df.columns:
    df['edad_meses'] = df['Edad'].apply(edad_a_meses)

In [103]:
def calcular_control_esperado(edad_meses):
    """
    Convierte la columna calculada DAX control_esperado a Python
    
    Parámetros:
    edad_meses: Serie de pandas o valor individual con la edad en meses
    
    Retorna:
    Serie de pandas o valor con el control esperado calculado
    """
    
    def control_esperado_individual(mt):
        """
        Función para calcular control esperado para un valor individual
        """
        # Manejar valores nulos/NaN
        if pd.isna(mt):
            return np.nan
        
        # SWITCH logic equivalente
        if mt < 12:
            # mensual 0-<12m → 1..11
            return int(mt) + 1
        elif mt < 24:
            # 12-<24m → cada 2m (12..17)
            return 11 + int((mt - 12) / 2) + 1
        elif mt < 36:
            # 24-<36m → cada 3m (18..21)
            return 17 + int((mt - 24) / 3) + 1
        elif mt < 48:
            # 36-<48m → cada 3m (22..25)
            return 21 + int((mt - 36) / 3) + 1
        else:
            # ≥48m → alta
            return 26
    
    # Si es una Serie de pandas, aplicar la función a cada elemento
    if isinstance(edad_meses, pd.Series):
        return edad_meses.apply(control_esperado_individual)
    else:
        # Si es un valor individual
        return control_esperado_individual(edad_meses)

In [104]:
df['control_esperado'] = calcular_control_esperado(df['edad_meses'])

In [105]:
df.head()

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc,edad_meses,control_esperado
0,2014-10-01,57921.0,CRED,N,M,1m,1.0,4.18,53.4,37.0,Normal,+-1,+-1,-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,EP,NaN,SI,SI,SI,1.0,2
1,2014-11-06,57921.0,CRED,C,M,2m,2.0,5.53,57.6,38.0,Normal,+-1,+-1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,8,EP,NaN,SI,SI,SI,2.0,3
2,2014-12-03,57921.0,CRED,C,M,3m,3.0,6.32,61.2,39.0,Normal,+1,+1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,Recup,NaN,SI,SI,SI,3.0,4
3,2015-01-05,57921.0,CRED,C,M,4m,4.0,7.09,64.0,39.5,Normal,+-1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI,4.0,5
4,2015-02-02,57921.0,CRED,C,M,5m,5.0,7.61,66.1,NaN,Normal,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,SI,NaN,SI,SI,SI,5.0,6


In [106]:
def calcular_flg_desarrollo(df, columna_desarrollo='(C) - Cog'):
    """
    Evalúa diferentes condiciones sobre la columna cognitiva:
    - Si es vacío/nulo ("") → BLANK (NaN)
    - Si es "Defic" → 1 (déficit detectado)
    - Si no está vacío → 0 (tiene valor pero no es déficit)
    - Si es diferente de vacío → 0
    - Default → BLANK (NaN)
    
    Parámetros:
    df: DataFrame que contiene la columna del tipo de desarrollo
    columna_desarrollo: nombre de la columna cognitiva (por defecto '(C) - Cog')
    
    Retorna:
    Serie de pandas con los flags 
    """
    
    # Verificar que la columna existe
    if columna_desarrollo not in df.columns:
        raise ValueError(f"La columna '{columna_desarrollo}' no existe en el DataFrame")
    
    def evaluar_desarrollo(valor):
        # Convertir a string para comparaciones, manejar NaN
        if pd.isna(valor):
            return np.nan  # BLANK()
        
        valor_str = str(valor).strip()  # Convertir a string y quitar espacios
        
        # Lógica del SWITCH:
        if valor_str == "":
            # DESARROLLO[(C) - Cog] = "" → BLANK()
            return np.nan
        elif valor_str == "Defic":
            # DESARROLLO[(C) - Cog] = "Defic" → 1
            return 1
        elif valor_str != "":
            # not(ISBLANK(DESARROLLO[(C) - Cog])) → 0
            # (es decir, si no está vacío pero tampoco es "Defic")
            return 0
        else:
            # Default case → BLANK()
            return np.nan
    
    # Aplicar la función a toda la serie
    resultado = df[columna_desarrollo].apply(evaluar_desarrollo)
    
    return resultado

In [107]:
df['flg_cognitivo']=calcular_flg_desarrollo(df,'(C) - Cog')
df['flg_lenguaje']=calcular_flg_desarrollo(df,'(L) - Len')
df['flg_motora_fina']=calcular_flg_desarrollo(df,'(M) - FF')
df['flg_motora_gruesa']=calcular_flg_desarrollo(df,'(M) - FG')
df['flg_social']=calcular_flg_desarrollo(df,'(S) - Soc')

In [108]:
def calcular_flg_alguna(df):
    
    # Lista de columnas de flags a sumar
    columnas_flags = [
        'flg_cognitivo',
        'flg_lenguaje', 
        'flg_motora_fina',
        'flg_motora_gruesa',
        'flg_social'
    ]
    
    suma_flags = df[columnas_flags].sum(axis=1)
    
    # Evaluar si la suma es mayor a 0 (equivalente a if(_r > 0, 1, _r))
    # Esto significa: si suma > 0 entonces 1, sino el valor de la suma
    flg_alguna = np.where(suma_flags > 0, 1, suma_flags)
    
    return pd.Series(flg_alguna, index=df.index)

In [109]:
df['flg_alguna'] = calcular_flg_alguna(df)

In [110]:
def calcular_flg_total(df):
    columnas_flags = [
        'flg_cognitivo',
        'flg_lenguaje', 
        'flg_motora_fina',
        'flg_motora_gruesa',
        'flg_social'
    ]
    flg_total = df[columnas_flags].sum(axis=1, skipna=True)
    
    return flg_total

In [111]:
df['flg_total'] = calcular_flg_total(df)

In [112]:
def calcular_primer_alguna_por_hc(df):
    """
    busca el primer control esperado donde hay algún déficit detectado (flg_alguna = 1) para cada HC.
    """
    columnas_requeridas = ['N_HC', 'flg_alguna', 'control_esperado']
    columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
    if columnas_faltantes:
        raise ValueError(f"Columnas faltantes: {columnas_faltantes}")
    
    def primer_alguna_grupo(grupo_df):
        """Función para aplicar a cada grupo de HC"""
        # Filtrar solo registros con flg_alguna = 1
        registros_deficit = grupo_df[grupo_df['flg_alguna'] == 1]
        
        if registros_deficit.empty:
            return np.nan
        
        # Retornar el primer control_esperado
        return registros_deficit['control_esperado'].min()
    
    # Aplicar por grupo de HC
    resultado = df.groupby('N_HC').apply(primer_alguna_grupo)
    
    return resultado

In [113]:
df['primer_alguna'] = calcular_primer_alguna_por_hc(df)

C:\Users\isma_\AppData\Local\Temp\ipykernel_18028\4109820522.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultado = df.groupby('N_HC').apply(primer_alguna_grupo)


In [117]:
df.shape

(454901, 43)

In [115]:
def calcular_primer_control_esperado(df):
    columnas_requeridas = ['N_HC', 'control_esperado']
    columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
    if columnas_faltantes:
        raise ValueError(f"Columnas faltantes: {columnas_faltantes}")
    
    # Usar transform para mantener el índice original
    resultado = df.groupby('N_HC')['control_esperado'].transform('min')
    
    return resultado

In [116]:
df['primer_control_esperado']=calcular_primer_control_esperado(df)

In [118]:
def calcular_ultimo_control(df):
    columnas_requeridas = ['N_HC', 'control_esperado']
    columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
    if columnas_faltantes:
        raise ValueError(f"Columnas faltantes: {columnas_faltantes}")
    
    # Calcular el último control esperado por cada HC
    ultimo_control_por_hc = df.groupby('N_HC')['control_esperado'].max()
    
    return ultimo_control_por_hc

In [119]:
df['ultimo_control']=calcular_ultimo_control(df)

In [123]:
def cant_controles_primer_alguna_por_hc(df):

    columnas_requeridas = ['N_HC', 'control_esperado', 'primer_alguna', 'ultimo_control']
    columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
    if columnas_faltantes:
        raise ValueError(f"Columnas faltantes: {columnas_faltantes}")
    
    def calcular_para_hc(grupo_df):
        """Función para aplicar a cada grupo de HC"""
        # Variable _alguna
        primer_alguna_val = grupo_df['primer_alguna'].iloc[0]
        if pd.notna(primer_alguna_val):
            _alguna = (grupo_df['control_esperado'] < primer_alguna_val).sum()
        else:
            _alguna = np.nan
        
        # Variable _otros
        ultimo_control_val = grupo_df['ultimo_control'].iloc[0]
        if pd.notna(ultimo_control_val):
            _otros = (grupo_df['control_esperado'] < ultimo_control_val).sum()
        else:
            _otros = np.nan
        
        # COALESCE
        if pd.notna(_alguna) and _alguna > 0:
            return _alguna
        elif pd.notna(_otros):
            return _otros
        else:
            return 0
    
    # Aplicar por grupo de HC
    resultado = df.groupby('N_HC').apply(calcular_para_hc)
    
    return resultado

In [124]:
df['cant_controles_primer_alguna'] = cant_controles_primer_alguna_por_hc(df)

C:\Users\isma_\AppData\Local\Temp\ipykernel_18028\4237762130.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultado = df.groupby('N_HC').apply(calcular_para_hc)


In [125]:
def calcular_cantidad_controles_por_hc(df):
    # Contar registros por cada HC
    cantidad_por_hc = df['N_HC'].value_counts().sort_index()
    
    return cantidad_por_hc

In [126]:
df['cantidad_controles']=calcular_cantidad_controles_por_hc(df)

In [127]:
df.head()

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc,edad_meses,control_esperado,flg_cognitivo,flg_lenguaje,flg_motora_fina,flg_motora_gruesa,flg_social,flg_alguna,flg_total,primer_alguna,primer_control_esperado,ultimo_control,cant_controles_primer_alguna,cantidad_controles
0,2014-10-01,57921.0,CRED,N,M,1m,1.0,4.18,53.4,37.0,Normal,+-1,+-1,-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,EP,NaN,SI,SI,SI,1.0,2,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,20.0,0.0,4.0
1,2014-11-06,57921.0,CRED,C,M,2m,2.0,5.53,57.6,38.0,Normal,+-1,+-1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,8,EP,NaN,SI,SI,SI,2.0,3,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN
2,2014-12-03,57921.0,CRED,C,M,3m,3.0,6.32,61.2,39.0,Normal,+1,+1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,Recup,NaN,SI,SI,SI,3.0,4,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN
3,2015-01-05,57921.0,CRED,C,M,4m,4.0,7.09,64.0,39.5,Normal,+-1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI,4.0,5,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN
4,2015-02-02,57921.0,CRED,C,M,5m,5.0,7.61,66.1,NaN,Normal,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,SI,NaN,SI,SI,SI,5.0,6,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN


In [ ]:
def num_controles_previos_deficit_por_hc(df):
    """
    Calcula la distancia entre primer_alguna y primer_Control_esperado para cada HC
    """
    
    columnas_requeridas = ['N_HC', 'primer_alguna', 'primer_control_esperado']
    columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
    if columnas_faltantes:
        raise ValueError(f"Columnas faltantes: {columnas_faltantes}")
    
    def calcular_para_hc(grupo_df):
        primer_alguna = grupo_df['primer_alguna'].iloc[0]
        primer_control = grupo_df['primer_control_esperado'].iloc[0]
        
        if pd.notna(primer_alguna) and pd.notna(primer_control):
            return primer_alguna - primer_control
        else:
            return np.nan
    
    resultado = df.groupby('N_HC').apply(calcular_para_hc)
    
    return resultado

In [ ]:
df['num_controles_previos_deficit']=num_controles_previos_deficit_por_hc(df)

C:\Users\isma_\AppData\Local\Temp\ipykernel_18028\179628090.py:20: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultado = df.groupby('N_HC').apply(calcular_para_hc)


In [130]:
def calcular_nro_control_recreado_por_hc(df):
    """
    Asigna un número de ranking (1, 2, 3...) basado en la fecha,
    ordenado ascendentemente para cada HC por separado.
    """
    
    # Asegurar que Fecha esté en formato datetime
    df_copia = df.copy()
    df_copia['Fecha'] = pd.to_datetime(df_copia['Fecha'])
    
    # Crear ranking por HC ordenado por fecha (ASC = ascendente)
    # method='dense' para que no haya gaps en la numeración
    nro_control = df_copia.groupby('N_HC')['Fecha'].rank(method='dense', ascending=True)
    
    # Convertir a enteros
    nro_control = nro_control.astype(int)
    
    return nro_control

In [131]:
df['nro_control_recreado']=calcular_nro_control_recreado_por_hc(df)

In [132]:
df.head()

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc,edad_meses,control_esperado,flg_cognitivo,flg_lenguaje,flg_motora_fina,flg_motora_gruesa,flg_social,flg_alguna,flg_total,primer_alguna,primer_control_esperado,ultimo_control,cant_controles_primer_alguna,cantidad_controles,distancia_primer_control_alguna,nro_control_recreado
0,2014-10-01,57921.0,CRED,N,M,1m,1.0,4.18,53.4,37.0,Normal,+-1,+-1,-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,EP,NaN,SI,SI,SI,1.0,2,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,20.0,0.0,4.0,NaN,1
1,2014-11-06,57921.0,CRED,C,M,2m,2.0,5.53,57.6,38.0,Normal,+-1,+-1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,8,EP,NaN,SI,SI,SI,2.0,3,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN,NaN,2
2,2014-12-03,57921.0,CRED,C,M,3m,3.0,6.32,61.2,39.0,Normal,+1,+1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,Recup,NaN,SI,SI,SI,3.0,4,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN,NaN,3
3,2015-01-05,57921.0,CRED,C,M,4m,4.0,7.09,64.0,39.5,Normal,+-1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI,4.0,5,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN,NaN,4
4,2015-02-02,57921.0,CRED,C,M,5m,5.0,7.61,66.1,NaN,Normal,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,SI,NaN,SI,SI,SI,5.0,6,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN,NaN,5


In [137]:
def num_controles_posteriores_deficit_por_nhc(df):
    """cuenta cuántos controles tuvo desde que se detectó el primer problema"""
    
    def calc_por_hc(grupo):
        primer_alguna = grupo['primer_alguna'].iloc[0]
        if pd.notna(primer_alguna):
            return (primer_alguna <= grupo['control_esperado']).sum()
        else:
            return np.nan
    
    return df.groupby('N_HC').apply(calc_por_hc)

In [138]:
df['num_controles_posteriores_deficit']=num_controles_posteriores_deficit_por_nhc(df)

C:\Users\isma_\AppData\Local\Temp\ipykernel_18028\367931374.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('N_HC').apply(calc_por_hc)


Limpieza de variables 

In [143]:
def parse_z(x, eps=1e-6):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().upper()

    # Casos especiales que se mantienen tal cual
    codigos_especiales = {"DA", "DC", "DG", "N", "NB", "NN", "O", "R", "S", "SA"}
    if s in codigos_especiales:
        return s  # mantener como está
        
    sin_dato_set = {"S/A", "SIN DATO", "NA", "N/A", ""}
    if s in sin_dato_set:
        return np.nan

    # Normalizar separadores y signos "raros"
    s = s.replace(",", ".")          # coma -> punto decimal
    s = s.replace(" ", "")           # espacios
    s = s.replace("+-", "")          # +- -> 
    s = s.replace("++", "+")         # ++ -> +
    s = s.replace("--", "-")         # - -> -
    s = s.replace("+>", ">+")
    s = s.replace("+<", "<+")
    s = s.replace("1+", "+1")        # 1+ -> +1 

    op = None
    if s.startswith(">"):
        op = ">"
        s = s[1:]
    elif s.startswith("<"):
        op = "<"
        s = s[1:]

   # Quitar un "+", +1" -> "1"
    if s.startswith("+"):
        s = s[1:]

    # Extraer número (permite signo y decimales)
    m = re.search(r"^[+-]?\d+(\.\d+)?$", s)
    if not m:
        # Valida si se limpio
        s2 = re.sub(r"[^\d\.\-+]", "", s)
        m2 = re.search(r"^[+-]?\d+(\.\d+)?$", s2)
        if not m2:
            return np.nan
        s = s2

    try:
        val = float(s)
    except:
        return np.nan

    # Respetar > y < desplazando un epsilon
    if op == ">":
        val = val + eps
    elif op == "<":
        val = val - eps

    return val

#categorizacion
def categoria_PE(z):
    if isinstance(z, str):   # conservar códigos especiales
        return z
    """P/E: DESN (< -2) | N [-2, +2] | SOBREPESO (> +2)"""
    if pd.isna(z):
        return "NULL"
    if z < -2:
        return "R"
    if z <= 2:
        return "N"
    return "S"  # z > +2

def categoria_PT(z):
    if isinstance(z, str):   # conservar códigos especiales
        return z
    if pd.isna(z):
        return "NULL"
    if z < -3:
        return "DA"
    if z < -2:
        return "R"
    if z < -1:
        return "N"
    if z < 1:
        return "N"
    if z < 2:
        return "N"
    if z < 3:
        return "S"
    return "O"  # z >= +3

In [144]:
# Crear columnas numéricas limpias (z-score) sin alterar las originales
df["_TE_z"] = df["T/E"].apply(parse_z)
df["_PE_z"] = df["P/E"].apply(parse_z)
df["_PT_z"] = df["P/T"].apply(parse_z)

# Crear las 3 columnas categóricas nuevas
#df["T/E_cat"] = df["_TE_z"].apply(categoria_TE)
df["P/E_cat"] = df["_PE_z"].apply(categoria_PE)
df["P/T_cat"] = df["_PT_z"].apply(categoria_PT)

In [145]:
df[["T/E","P/E","P/T","P/E_cat","P/T_cat"]].head() #,"T/E_cat" FALTA

,T/E,P/E,P/T,P/E_cat,P/T_cat
0,+-1,-1,+-1,N,N
1,+-1,+-1,+-1,N,N
2,+1,+-1,+1,N,N
3,+1,+1,+-1,N,N
4,+1,+1,+1,N,N


### Outliers

In [146]:
def detectar_outliers_iqr(df, col, factor=1.5):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - factor * IQR
    upper = Q3 + factor * IQR
    return df[(df[col] < lower) | (df[col] > upper)]

out_peso_iqr = detectar_outliers_iqr(df, 'Peso').sort_values('Peso', ascending=False)
print("Outliers en Peso (IQR):", len(out_peso_iqr))

out_talla_iqr = detectar_outliers_iqr(df, 'Talla').sort_values('Talla', ascending=False)
print("Outliers en Talla (IQR):", len(out_talla_iqr))

out_talla_iqr = detectar_outliers_iqr(df, 'CabPC').sort_values('CabPC', ascending=False)
print("Outliers en Talla (IQR):", len(out_talla_iqr))

Outliers en Peso (IQR): 9996
Outliers en Talla (IQR): 1076
Outliers en Talla (IQR): 161


In [148]:
out_peso_iqr[['N_HC','edad_meses','Talla','Fecha','Peso']].head(20)

,N_HC,edad_meses,Talla,Fecha,Peso
23094,46963.0,20.0,81.1,2013-10-22,130020.0
173817,49308.0,4.0,59.7,2013-01-08,73560.0
283261,73636.0,44.0,103.1,2021-12-29,16600.0
100875,36063.0,42.0,93.4,2013-02-06,14100.0
413260,72286.0,32.0,89.8,2020-09-25,13700.0
386537,71803.0,33.0,87.5,2020-08-31,13300.0
134623,51502.0,22.0,86.8,2015-01-02,13090.0
81857,47732.0,30.0,92.5,2014-10-25,13090.0
345527,77853.0,14.0,78.7,2020-09-07,12360.0
48049,52726.0,14.0,75.3,2014-08-20,10250.0


In [149]:
# Imputacion por el percentile 99
for col in ['Peso','Talla','CabPC']:
    lower = df[col].quantile(0.001)
    upper = df[col].quantile(0.999)
    df[col] = np.clip(df[col], lower, upper)

In [150]:
df.head()

,Fecha,N_HC,Tipo_Paciente,Categoría,Sexo,Edad,N_Control,Peso,Talla,CabPC,Diag_Nacimiento,P/T,T/E,P/E,Ganancia_Peso_Talla,Dx_Nutricional,CN-CA,Mantiene_Diag_Fav/Desf,Recuperado,Lactancia,Razón,ACA,Tam_para,Tam_graha,Tam_hb,Acompaña_control,Enfermera,(M) - FG,(M) - FF,(C) - Cog,(L) - Len,(S) - Soc,edad_meses,control_esperado,flg_cognitivo,flg_lenguaje,flg_motora_fina,flg_motora_gruesa,flg_social,flg_alguna,flg_total,primer_alguna,primer_control_esperado,ultimo_control,cant_controles_primer_alguna,cantidad_controles,nro_control_recreado,num_controles_previos_deficit,num_controles_posteriores_deficit,_TE_z,_PE_z,_PT_z,P/E_cat,P/T_cat
0,2014-10-01,57921.0,CRED,N,M,1m,1.0,4.18,53.4,37.0,Normal,+-1,+-1,-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,EP,NaN,SI,SI,SI,1.0,2,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,20.0,0.0,4.0,1,NaN,NaN,1.0,-1.0,1.0,N,N
1,2014-11-06,57921.0,CRED,C,M,2m,2.0,5.53,57.6,38.0,Normal,+-1,+-1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,8,EP,NaN,SI,SI,SI,2.0,3,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN,2,NaN,NaN,1.0,1.0,1.0,N,N
2,2014-12-03,57921.0,CRED,C,M,3m,3.0,6.32,61.2,39.0,Normal,+1,+1,+-1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,Recup,NaN,SI,SI,SI,3.0,4,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN,3,NaN,NaN,1.0,1.0,1.0,N,N
3,2015-01-05,57921.0,CRED,C,M,4m,4.0,7.09,64.0,39.5,Normal,+-1,+1,+1,GAP-GAT,Normal,NE,Fav,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,2,SI,NaN,SI,SI,SI,4.0,5,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN,4,NaN,NaN,1.0,1.0,1.0,N,N
4,2015-02-02,57921.0,CRED,C,M,5m,5.0,7.61,66.1,NaN,Normal,+1,+1,+1,GAP-GAT,Normal,NaN,NaN,NaN,LME,NaN,NaN,NaN,NaN,NaN,Mamá,3,SI,NaN,SI,SI,SI,5.0,6,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,2,NaN,NaN,NaN,5,NaN,NaN,1.0,1.0,1.0,N,N


Se exporta tabla para segmentacion y feature engineering

In [151]:
df.to_csv("../data/intermediate/tani_preprocessed_1.csv", index=False)